In [42]:
import pandas as pd
import numpy as np
import cv2
import sys
import importlib
SEED = 1234
np.random.seed(SEED) 

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, AveragePooling2D, Concatenate, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l1, l2
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from mylibs.VggNet import VGG16
import mylibs.ResNet as ResNet
import mylibs.SENet as SENet
importlib.reload(ResNet)
importlib.reload(SENet)
from keras.models import Model

from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit, train_test_split
import matplotlib.pyplot as plt
from scipy.ndimage.filters import uniform_filter

In [2]:
%cd E:\kaggle\iceberg

E:\kaggle\iceberg


In [3]:
def display_img(band_1, band_2, is_iceberg, angle = None):
    if angle is None:
        title_str = 'Iceberg' if is_iceberg == 1 else 'Ship'
    else:
        title_str = 'Iceberg-' + str(angle) if is_iceberg == 1 else 'Ship-' + str(angle)
    fig = plt.figure(0, figsize=(10,10))
    ax = fig.add_subplot(1,2,1)
    ax.set_title(title_str + ' - Band 1')
    ax.imshow(band_1,cmap='jet')
    ax = fig.add_subplot(1,2,2)
    ax.set_title(title_str + ' - Band 2')
    ax.imshow(band_2,cmap='jet')
    plt.show()

# implement functions to convert SAR data from decibel units to linear units and back again
def decibel_to_linear(band):
     # convert to linear units
    return np.power(10,np.array(band)/10)

def linear_to_decibel(band):
    return 10*np.log10(band)

# implement the Lee Filter for a band in an image already reshaped into the proper dimensions
def lee_filter(band, window, var_noise = 0.25):
    # band: SAR data to be despeckled (already reshaped into image dimensions)
    # window: descpeckling filter window (tuple)
    # default noise variance = 0.25
    # assumes noise mean = 0
    
    mean_window = uniform_filter(band, window)
    mean_sqr_window = uniform_filter(band**2, window)
    var_window = mean_sqr_window - mean_window**2

    weights = var_window / (var_window + var_noise)
    band_filtered = mean_window + weights*(band - mean_window)
    return band_filtered

def apply_lee_filter(band_1_linear, band_2_linear, window_var_index = 0, noise_var_index = 0):
    windows = [2, 4, 8] # can be tuple too if not symetric
    noise_var = np.array([1, 2, 4])
    noise_var_1 = np.round(np.var(band_1_linear) * noise_var, 10)
    noise_var_2 = np.round(np.var(band_2_linear) * noise_var, 10)
    band_1_linear_filtered = lee_filter(band_1_linear, windows[window_var_index], noise_var_1[noise_var_index])
    band_2_linear_filtered = lee_filter(band_2_linear, windows[window_var_index], noise_var_2[noise_var_index])
    return band_1_linear_filtered, band_2_linear_filtered

def apply_lee_filter_single(band_linear, window_var_index = 0, noise_var_index = 0):
    windows = [2, 4, 8] # can be tuple too if not symetric
    noise_var = np.array([1, 2, 4])
    noise_var = np.round(np.var(band_linear) * noise_var, 10)
    band_linear_filtered = lee_filter(band_linear, windows[window_var_index], noise_var[noise_var_index])
    return band_linear_filtered

In [4]:
def np_get_scaled_band(band_list):
    imgs = []
    for band in band_list:        
        imgs.append((band - band.mean()) / (band.max() - band.min()))
#         imgs.append(band - band.mean())
#         imgs.append((band - band.mean()) / band.std())
#         imgs.append(cv2.normalize(band, None, -1, 1, norm_type=cv2.NORM_MINMAX))
    return np.array(imgs)

def get_more_images(imgs):
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
    vh_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        vert_flip_imgs.append(cv2.flip(imgs[i], 1))
        hori_flip_imgs.append(cv2.flip(imgs[i], 0))
        vh_flip_imgs.append(cv2.flip(imgs[i], -1))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
    vh = np.array(vh_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h, vh))
    
    return more_images

In [5]:
def display_img(band_1, band_2, is_iceberg, angle = None):
    if angle is None:
        title_str = 'Iceberg' if is_iceberg == 1 else 'Ship'
    else:
        title_str = 'Iceberg-' + str(angle) if is_iceberg == 1 else 'Ship-' + str(angle)
    fig = plt.figure(0, figsize=(10,10))
    ax = fig.add_subplot(1,2,1)
    ax.set_title(title_str + ' - Band 1')
    ax.imshow(band_1,cmap='jet')
    ax = fig.add_subplot(1,2,2)
    ax.set_title(title_str + ' - Band 2')
    ax.imshow(band_2,cmap='jet')
    plt.show()

# implement functions to convert SAR data from decibel units to linear units and back again
def decibel_to_linear(band):
     # convert to linear units
    return np.power(10,np.array(band)/10)

def linear_to_decibel(band):
    return 10*np.log10(band)

# implement the Lee Filter for a band in an image already reshaped into the proper dimensions
def lee_filter(band, window, var_noise = 0.25):
    # band: SAR data to be despeckled (already reshaped into image dimensions)
    # window: descpeckling filter window (tuple)
    # default noise variance = 0.25
    # assumes noise mean = 0
    
    mean_window = uniform_filter(band, window)
    mean_sqr_window = uniform_filter(band**2, window)
    var_window = mean_sqr_window - mean_window**2

    weights = var_window / (var_window + var_noise)
    band_filtered = mean_window + weights*(band - mean_window)
    return band_filtered

def apply_lee_filter(band_1_linear, band_2_linear, window_var_index = 0, noise_var_index = 0):
    windows = [2, 4, 8] # can be tuple too if not symetric
    noise_var = np.array([1, 2, 4])
    noise_var_1 = np.round(np.var(band_1_linear) * noise_var, 10)
    noise_var_2 = np.round(np.var(band_2_linear) * noise_var, 10)
    band_1_linear_filtered = lee_filter(band_1_linear, windows[window_var_index], noise_var_1[noise_var_index])
    band_2_linear_filtered = lee_filter(band_2_linear, windows[window_var_index], noise_var_2[noise_var_index])
    return band_1_linear_filtered, band_2_linear_filtered

def apply_lee_filter_single(band_linear, window_var_index = 0, noise_var_index = 0):
    windows = [2, 4, 8] # can be tuple too if not symetric
    noise_var = np.array([1, 2, 4])
    noise_var = np.round(np.var(band_linear) * noise_var, 10)
    band_linear_filtered = lee_filter(band_linear, windows[window_var_index], noise_var[noise_var_index])
    return band_linear_filtered

In [6]:
train = pd.read_json("E:/kaggle/iceberg/train.json/data/processed/train.json")
test = pd.read_json("E:/kaggle/iceberg/test.json/data/processed/test.json")
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')

In [7]:
target_train=np.array(train['is_iceberg'])
train['inc_angle']=train['inc_angle'].fillna(method='pad')
test['inc_angle']=test['inc_angle'].fillna(method='pad')
# train["inc_angle"] = train["inc_angle"].replace('na',0)
# idx_tr = np.where(train["inc_angle"]>0)
# train = train.iloc[idx_tr[0]]
# target_train = target_train.iloc[idx_tr[0]]
X_angle=train['inc_angle']
X_test_angle=test['inc_angle']

In [8]:
#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
#apply filter
X_band_1_filtered = np.array([apply_lee_filter_single(decibel_to_linear(band)) for band in X_band_1])
X_band_2_filtered = np.array([apply_lee_filter_single(decibel_to_linear(band)) for band in X_band_2])
X_band_1_filtered = linear_to_decibel(X_band_1_filtered)
X_band_2_filtered = linear_to_decibel(X_band_2_filtered)
X_band_1 = X_band_1_filtered
X_band_2 = X_band_2_filtered

X_band_3=np.fabs(np.subtract(X_band_1,X_band_2))
X_band_4=np.maximum(X_band_1,X_band_2)
X_band_5=np.minimum(X_band_1,X_band_2)

X_band_3 = np_get_scaled_band(X_band_3)
X_band_4 = np_get_scaled_band(X_band_4)
X_band_5 = np_get_scaled_band(X_band_5)

X_train = np.concatenate([X_band_3[:, :, :, np.newaxis],X_band_4[:, :, :, np.newaxis],X_band_5[:, :, :, np.newaxis]], axis=-1)

X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
#apply filter
X_band_test_1_filtered = np.array([apply_lee_filter_single(decibel_to_linear(band)) for band in X_band_test_1])
X_band_test_2_filtered = np.array([apply_lee_filter_single(decibel_to_linear(band)) for band in X_band_test_2])
X_band_test_1_filtered = linear_to_decibel(X_band_test_1_filtered)
X_band_test_2_filtered = linear_to_decibel(X_band_test_2_filtered)
X_band_test_1 = X_band_test_1_filtered
X_band_test_2 = X_band_test_2_filtered

X_band_test_3=np.fabs(np.subtract(X_band_test_1,X_band_test_2))
X_band_test_4=np.maximum(X_band_test_1,X_band_test_2)
X_band_test_5=np.minimum(X_band_test_1,X_band_test_2)

X_band_test_3 = np_get_scaled_band(X_band_test_3)
X_band_test_4 = np_get_scaled_band(X_band_test_4)
X_band_test_5 = np_get_scaled_band(X_band_test_5)

X_test = np.concatenate([X_band_test_3[:, :, :, np.newaxis], X_band_test_4[:, :, :, np.newaxis],X_band_test_5[:, :, :, np.newaxis]],axis=-1)

Xtrain = X_train
Ytrain = target_train
Xtest = X_test
Xangle = X_angle
Xangle_test = X_test_angle
df_train = train
df_test = test

In [9]:
# resize_shape = tuple(np.array(Xtrain.shape[1:3]) * 2)
# Xtrain = np.array([cv2.resize(img, resize_shape) for img in Xtrain])
# Xtest = np.array([cv2.resize(img, resize_shape) for img in Xtest])

In [10]:
print(Xtrain.shape, Ytrain.shape, Xangle.shape, Xtest.shape)

(1604, 75, 75, 3) (1604,) (1604,) (8424, 75, 75, 3)


In [41]:
def getModel():
    angle_input = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(angle_input)
    vgg16_model = VGG16(weights=None, include_top=False, input_shape=Xtrain.shape[1:], pooling="max")
    mobile_model = MobileNet(weights=None, include_top=False, input_tensor = vgg16_model.input, input_shape=X_train.shape[1:], pooling="max")
#     incept_model = InceptionV3(include_top=False, input_tensor = vgg16_model.input, input_shape=Xtrain.shape[1:], pooling="max")
    xception_model = Xception(weights=None, include_top=False, input_tensor = vgg16_model.input, input_shape=X_train.shape[1:], pooling="max")
    
    regularizer1 = l1(0.05)
    regularizer2 = l1(0.08)
    
    x1 = vgg16_model.output
#     x1 = Flatten()(x1)
#     x1 = Dropout(0.3)(x1)
    x1 = Concatenate()([x1, angle_layer])
    x1 = Dense(512, activation='relu', kernel_regularizer=regularizer1)(x1)
    x1 = Dropout(0.3)(x1)
    
    x2 = mobile_model.output
#     x2 = Flatten()(x2)
#     x2 = Dropout(0.3)(x2)
    x2 = Concatenate()([x2, angle_layer])
    x2 = Dense(512, activation='relu', kernel_regularizer=regularizer1)(x2)
    x2 = Dropout(0.3)(x2)
    
    x3 = xception_model.output
#     x3 = Flatten()(x3)
#     x3 = Dropout(0.3)(x3)
    x3 = Concatenate()([x3, angle_layer])
    x3 = Dense(512, activation='relu', kernel_regularizer=regularizer2)(x3)
    x3 = Dropout(0.5)(x3)
    
    x = Concatenate()([x1, x2, x3, angle_layer])
    x = Dropout(0.3)(x)
    predictions = Dense(1, activation='sigmoid', name='predictions')(x)
    model = Model(inputs=[vgg16_model.input, angle_input], outputs=predictions)
    optimizer = Adam(lr=1e-4)
    log_loss = keras.losses.binary_crossentropy
    log_loss.__name__ = "log_loss"
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', log_loss])
    
    return model

In [34]:
model = getModel()
model.summary()
plot_model(model, to_file="vgg16.png")

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 75, 75, 3)     0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 37, 37, 32)    864         input_4[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, 37, 37, 32)    128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, 37, 37, 32)    0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [32]:
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.2,
                         height_shift_range = 0.2,
                         channel_shift_range=0,
                         zoom_range = 0.5,
                         rotation_range = 15)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size, seed=SEED)
    genX2 = gen.flow(X1,X2, batch_size=batch_size, seed=SEED)
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        yield [X1i[0], X2i[1]], X1i[1]

def get_callbacks(filepath):
    es = EarlyStopping('val_loss', patience=30, mode="min")
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave, reduce_lr_loss]

In [ ]:
# K=3
# epochs = 150
# batch_size = 64
# Kfolds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=SEED).split(Xtrain, Ytrain))
# y_test_pred_log = 0
# for j, (train_idx, test_idx) in enumerate(Kfolds):
#     print('\n===================FOLD=',j)
#     Xtrain_cv = Xtrain[train_idx]
#     Ytrain_cv = Ytrain[train_idx]
#     Xangle_cv = Xangle[train_idx]
#     Xtrain_val = Xtrain[test_idx]
#     Ytrain_val = Ytrain[test_idx]
#     Xangle_val = Xangle[test_idx]
    
#     Xtrain_input = [Xtrain, Xangle]
#     Xval_input = [Xtrain_val, Xangle_val]
#     Xtest_input = [Xtest, Xangle_test]
    
#     model_file = 'vgg16_%s.hdf5' % j

#     model = getModel()
#     steps = np.ceil(len(Xtrain_cv) / batch_size) * 3
#     model.fit_generator(gen_flow_for_two_inputs(Xtrain_cv, Xangle_cv, Ytrain_cv), 
#                         steps_per_epoch=steps, epochs=epochs, verbose=1, shuffle=True, 
#                         callbacks=get_callbacks(model_file), validation_data=(Xval_input, Ytrain_val))
    
#     model.load_weights(filepath = model_file)    
    
#     train_score = model.evaluate(Xtrain_input, Ytrain, verbose=1)
#     val_score = model.evaluate(Xval_input, Ytrain_val, verbose=1)
#     print('Train score:', train_score[0], ' ;Train accuracy:', train_score[1])
#     print('Valid score:', val_score[0], ' ;Valid accuracy:', val_score[1])
#     y_test_pred_log += model.predict(Xtest_input, verbose=1).reshape(Xtest.shape[0])
    
# y_test_pred_log /= K

In [ ]:
epochs = 400
batch_size = 64
Xtrain_cv, Xtrain_val, Ytrain_cv, Ytrain_val, Xangle_cv, Xangle_val = train_test_split(Xtrain, Ytrain, Xangle, shuffle=True, test_size=0.3, random_state=SEED)
y_test_pred_log = 0
 
Xtrain_input = [Xtrain, Xangle]
Xval_input = [Xtrain_val, Xangle_val]
Xtest_input = [Xtest, Xangle_test]

model_file = 'vgg16_%s.hdf5' % 1

model = getModel()
steps = np.ceil(len(Xtrain_cv) / batch_size) * 3
model.fit_generator(gen_flow_for_two_inputs(Xtrain_cv, Xangle_cv, Ytrain_cv), 
                    steps_per_epoch=steps, epochs=epochs, verbose=1, shuffle=True, 
                    callbacks=get_callbacks(model_file), validation_data=(Xval_input, Ytrain_val))

model.load_weights(filepath = model_file)    

train_score = model.evaluate(Xtrain_input, Ytrain, verbose=1)
val_score = model.evaluate(Xval_input, Ytrain_val, verbose=1)
print('Train score:', train_score[0], ' ;Train accuracy:', train_score[1])
print('Valid score:', val_score[0], ' ;Valid accuracy:', val_score[1])
y_test_pred_log += model.predict(Xtest_input, verbose=1).reshape(Xtest.shape[0])

Epoch 1/400
54/54 [==============================] - 198s - loss: 3045.5428 - acc: 0.5205 - log_loss: 1.2430 - val_loss: 2726.8255 - val_acc: 0.4917 - val_log_loss: 0.7372
Epoch 2/400
54/54 [==============================] - 31s - loss: 2443.0998 - acc: 0.5801 - log_loss: 1.0130 - val_loss: 2159.9549 - val_acc: 0.4917 - val_log_loss: 0.7256
Epoch 3/400
54/54 [==============================] - 31s - loss: 1910.2131 - acc: 0.7037 - log_loss: 0.7179 - val_loss: 1662.6844 - val_acc: 0.4917 - val_log_loss: 0.7154
Epoch 4/400
54/54 [==============================] - 31s - loss: 1446.6253 - acc: 0.7350 - log_loss: 0.6250 - val_loss: 1234.1272 - val_acc: 0.4917 - val_log_loss: 0.6973
Epoch 5/400
54/54 [==============================] - 31s - loss: 1051.4915 - acc: 0.7571 - log_loss: 0.5677 - val_loss: 873.7345 - val_acc: 0.4917 - val_log_loss: 0.7147
Epoch 6/400
54/54 [==============================] - 32s - loss: 724.4256 - acc: 0.7708 - log_loss: 0.5154 - val_loss: 581.3531 - val_acc: 0.4917

54/54 [==============================] - 24s - loss: 1.6723 - acc: 0.8971 - log_loss: 0.2422 - val_loss: 1.7160 - val_acc: 0.8900 - val_log_loss: 0.2879
Epoch 51/400
53/54 [============================>.] - ETA: 0s - loss: 1.6684 - acc: 0.8979 - log_loss: 0.2382
Epoch 00050: reducing learning rate to 9.999999747378752e-06.
54/54 [==============================] - 28s - loss: 1.6697 - acc: 0.8971 - log_loss: 0.2392 - val_loss: 1.7658 - val_acc: 0.9025 - val_log_loss: 0.3202
Epoch 52/400
54/54 [==============================] - 30s - loss: 0.5988 - acc: 0.9085 - log_loss: 0.2081 - val_loss: 0.4498 - val_acc: 0.8963 - val_log_loss: 0.2738
Epoch 53/400
54/54 [==============================] - 24s - loss: 0.3656 - acc: 0.9162 - log_loss: 0.1998 - val_loss: 0.4557 - val_acc: 0.8983 - val_log_loss: 0.2935
Epoch 54/400
54/54 [==============================] - 24s - loss: 0.3574 - acc: 0.9182 - log_loss: 0.1965 - val_loss: 0.4560 - val_acc: 0.8983 - val_log_loss: 0.2937
Epoch 55/400
54/54 [====

54/54 [==============================] - 29s - loss: 0.2001 - acc: 0.9210 - log_loss: 0.1922 - val_loss: 0.3073 - val_acc: 0.8942 - val_log_loss: 0.2994
Epoch 96/400
54/54 [==============================] - 29s - loss: 0.1955 - acc: 0.9210 - log_loss: 0.1877 - val_loss: 0.3073 - val_acc: 0.8942 - val_log_loss: 0.2994
Epoch 97/400
54/54 [==============================] - 24s - loss: 0.1938 - acc: 0.9214 - log_loss: 0.1860 - val_loss: 0.3073 - val_acc: 0.8942 - val_log_loss: 0.2994
Epoch 98/400
54/54 [==============================] - 24s - loss: 0.1981 - acc: 0.9227 - log_loss: 0.1902 - val_loss: 0.3073 - val_acc: 0.8942 - val_log_loss: 0.2994
Epoch 99/400
54/54 [==============================] - 24s - loss: 0.1959 - acc: 0.9238 - log_loss: 0.1881 - val_loss: 0.3073 - val_acc: 0.8942 - val_log_loss: 0.2995
Epoch 100/400
54/54 [==============================] - 24s - loss: 0.1978 - acc: 0.9213 - log_loss: 0.1900 - val_loss: 0.3073 - val_acc: 0.8942 - val_log_loss: 0.2995
Epoch 101/400
54

In [ ]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': y_test_pred_log})
print(submission.count(), Xtest.shape[0])

submission.to_csv('submission-vgg16-%s.csv' % baseModelName, index=False)